In [171]:
import numpy as np
import pandas as pd
PATH = 'raw_download/admin, 203.csv'

In [172]:
df = pd.read_csv(PATH)

In [173]:
# remove unannotated rows
df = df[~pd.isnull(df['label'])]

In [174]:
# remove comments
df = df.drop(columns='Comments')

In [175]:
# split labels
df['label'] = df['label'].apply(lambda s: sorted(s.split('#')))

In [176]:
# check for invalid labels
metrics_expected = ['accuracy', 'fluency', 'code-mixedness']
invalid_idx = []
for i, l in enumerate(df['label'].tolist()):
    metrics = [
        score.split(' ')[1]
        for score in l
    ]
    if sorted(metrics_expected) != sorted(metrics): # either missing or duplicate metric labels
        invalid_idx.append(i)
            

pd.DataFrame(
    {
        'annotated_count': [df.shape[0]],
        'invalid_count': [len(invalid_idx)],
        'invalid_rate': [len(invalid_idx) / df.shape[0]]
    }
)

,annotated_count,invalid_count,invalid_rate
0,203,0,0.0


In [177]:
# extract scores
scores = pd.DataFrame([
    {
        metric: description
        for description, metric in (score.split() for score in l)
    }
        for l in df['label'].tolist()
]).map({
    'low': 1,
    'moderate': 2,
    'high': 3
}.get)


In [178]:
# assign scores to df
df[metrics_expected] = scores
df = df.drop(columns='label')

In [179]:
# add 'human' prefix
df = df.rename(
    columns={
        metric: f'human_{metric}'
        for metric in metrics_expected
    }
)

In [180]:
df

,id,text,src,tgt,human_accuracy,human_fluency,human_code-mixedness
0,8dcf43a1-4321-481e-994d-4e32773efa84,“而衣服湿漉漉，常常汗淋淋的孩子，通常可以在他身上看到皮肤红疹。”,"""And for children often with wet clothes and s...",“ 而 衣服 湿漉漉 ， 常 汗淋淋 的 孩子 ， 同时 常能 在 他 身上 看见 皮肤 红...,3,3,1
1,3a5227c0-9bb4-4d07-b1ae-2b6b24b5186e,"“ The clothes always 湿漉漉, and the kid is 常汗淋淋,...","""And for children often with wet clothes and s...",“ 而 衣服 湿漉漉 ， 常 汗淋淋 的 孩子 ， 同时 常能 在 他 身上 看见 皮肤 红...,3,3,3
2,694f4571-269e-42b7-a23b-5a465621ef10,""" Wah, dis kid always sweat a lot, leh? Clothe...","""And for children often with wet clothes and s...",“ 而 衣服 湿漉漉 ， 常 汗淋淋 的 孩子 ， 同时 常能 在 他 身上 看见 皮肤 红...,3,2,1
3,41b88393-9948-4431-abd0-b13230152e58,""" Ah, dat kpkb kid, always got wet-wet clothes...","""And for children often with wet clothes and s...",“ 而 衣服 湿漉漉 ， 常 汗淋淋 的 孩子 ， 同时 常能 在 他 身上 看见 皮肤 红...,3,2,1
4,183db7e5-1019-43fd-b114-bc71bdb44553,"""小朋友常常穿着湿湿的衣服和流汗，身上会常常发现皮肤 rash.""","""And for children often with wet clothes and s...",“ 而 衣服 湿漉漉 ， 常 汗淋淋 的 孩子 ， 同时 常能 在 他 身上 看见 皮肤 红...,3,2,2
...,...,...,...,...,...,...,...
198,15fbf0fb-e0d0-413c-87e4-6dc1103aae62,"He explain say, “Using body remote sensing tec...","""It was utterly impossible to adopt internal t...",他 解释 说 ， “ 采用 体内 遥测技术 曾经 是 绝无 可能 的 ， 因为 无法 将 一...,3,2,3
199,722d7203-7390-4265-af75-6a5d673a9d48,"""以前要用 internal telemetry technology 是完全不可能的，因为...","""It was utterly impossible to adopt internal t...",他 解释 说 ， “ 采用 体内 遥测技术 曾经 是 绝无 可能 的 ， 因为 无法 将 一...,3,3,2
200,5d275535-ce05-45ca-a9c6-9738d7a92ced,"He say, back then, no way leh, cannot put tiny...","""It was utterly impossible to adopt internal t...",他 解释 说 ， “ 采用 体内 遥测技术 曾经 是 绝无 可能 的 ， 因为 无法 将 一...,1,1,2
201,86b4d5c6-271a-4a14-bb64-18df02ff3539,"Lah, he say, ""Use internal remote sensing tech...","""It was utterly impossible to adopt internal t...",他 解释 说 ， “ 采用 体内 遥测技术 曾经 是 绝无 可能 的 ， 因为 无法 将 一...,2,1,1


In [181]:
df.to_csv(f'cleaned_up.csv', index=False)